In [ ]:
from daphne import daphne
import os

In [ ]:
directory = 'programs/tests/hoppl-deterministic'
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/{}/'.format(directory))

In [ ]:
i=3
cwd='/Users/gw/repos/daphne'
exp = daphne(['desugar-hoppl', '-i', '../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne'.format(i)])
exp



In [ ]:
desugared_ast_json_fname = '/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne.json'.format(i)
with open(desugared_ast_json_fname, 'w') as f:
    json.dump(exp, f)

In [ ]:
%ls /Users/gw/repos/daphne/../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic

In [ ]:
class Env():
    "An environment: a dict of {'var': val} pairs, with an outer Env."
    def __init__(self, parms=(), args=(), outer=None):
        self.data = pmap(zip(parms, args))
        self.outer = outer
        if outer is None:
            self.level = 0
        else:
            self.level = outer.level+1
    def __getitem__(self, item):
        return self.data[item]
    def find(self, var):
        "Find the innermost Env where var appears."
        if (var in self.data):
            return self
        else:
            if self.outer is not None:
                return self.outer.find(var)
            else:
                raise RuntimeError('var "{}" not found in outermost scope'.format(var))
    
    def print_env(self, print_lowest=False):
        print_limit = 1 if print_lowest == False else 0
        outer = self
        while outer is not None:
            if outer.level >= print_limit:
                print('Scope on level ', outer.level)
                if 'f' in outer:
                    print('Found f, ')
                    print(outer['f'].body)
                    print(outer['f'].parms)
                    print(outer['f'].env)
                print(outer,'\n')
            outer = outer.outer

In [ ]:
import copy

class Procedure(object):
    "A user-defined Scheme procedure."
    def __init__(self, parms, body, env):
        self.parms, self.body, self.env = parms, body, env
    def __call__(self, *args): 
        new_env = copy.deepcopy(self.env)
        return eval(self.body, Env(self.parms, args, new_env)) # [0]

In [ ]:
from pyrsistent import pmap

import torch
import torch.distributions as dist

In [ ]:
def push_addr(alpha, value):
    return alpha + value

penv = {
    '+': torch.add,
    '*':torch.mul,
#     'push-address':push_addr
}

In [ ]:
def standard_env():
    "An environment with some Scheme standard procedures."
    env = Env(penv.keys(), penv.values())
    return env

env = standard_env()
env.find('+')
#['+'](torch.tensor(0.),torch.tensor(1.))

In [ ]:
def eval(x,env=standard_env(),sigma=None):

    print('x',x)
    if isinstance(x,str):
        return env.find(x)[x], sigma
    elif not isinstance(x,list):
        return torch.tensor(x),sigma
    
    op, param, *args = x
    
    if False:
        pass
    
    elif op == 'push-address':
        return '', sigma
    elif op == 'fn':
        print('args',args)
#         param, body = args
        body = args[0]
        return Procedure(param, body, env), sigma # has eval in it
        # param ['alpha', 'x']
        # body ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']
        # env
    
    else:
        print('in else. x',x)
        proc, _ = eval(op,env,sigma)
        vals = [''] # TODO: address for fn???
        vals.extend([eval(arg, env, sigma)[0] for arg in args])
        print('vals',vals)

        if isinstance(proc, Procedure): # lambdas, not primitives
            print('in Procedure',proc, vals)
            r, _ = proc(*vals)
        else:
            r = proc(*vals[1:]) # primitives
            print('in primitives',proc, vals)
            
        return r, sigma
    

# eval(['*','a0',3,3])  

# why vals only tensor(8.), where did push add go?

In [ ]:
exp

In [ ]:
proc, sigma = eval(exp)
proc('')

In [ ]:
exp_sq_8 = [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']],
   ['push-address', 'alpha', 'addr1'],
   9]

eval(exp_sq_8)#(torch.tensor(8.))

In [ ]:
exp_inner = ['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
eval(exp_inner)#(torch.tensor(8.))

In [ ]:
import sys
sys.setrecursionlimit(10000)

# test driven development

# deterministic-hoppl test 3

In [43]:
from tests import load_truth
import os, json
from daphne import daphne

import importlib
import evaluator
importlib.reload(evaluator)
from evaluator import evaluate


In [51]:
def ast_helper(fname,directory='programs'):
    os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/{}/'.format(directory))
    sugared_fname = '../prob_prog/hw/hw5/CS532-HW5/{}/{}'.format(directory,fname)
    desugared_ast_json_fname = '/Users/gw/repos/prob_prog/' + sugared_fname.replace('.daphne','.daphne.json')
    if os.path.isfile(desugared_ast_json_fname):
        with open(desugared_ast_json_fname) as f:
            ast = json.load(f)
    else:
        #note: the sugared path that goes into daphne desugar should be with respect to the daphne path!
        ast = daphne(['desugar-hoppl', '-i', sugared_fname]) 
        #exp = daphne(['desugar-hoppl', '-i', '../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne'.format(i)])

        with open(desugared_ast_json_fname, 'w') as f:
            json.dump(ast, f)
    return ast

i=2
fname='test_{}.daphne'.format(i)
exp = ast_helper(fname,directory='programs/tests/deterministic')

['fn', ['alpha'], ['sqrt', ['push-address', 'alpha', 'addr0'], 2]]

In [54]:
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/')
truth = load_truth('programs/tests/deterministic/test_{}.truth'.format(i))
truth

1.4142136

In [53]:
fname = 'programs/tests/deterministic/test_{}.daphne.json'.format(i)

with open(fname) as f:
     exp = json.load(f)
exp
# 

['fn', ['alpha'], ['+', ['push-address', 'alpha', 'addr0'], 5, 2]]

In [50]:
ret = evaluate(exp)
ret

x ['fn', ['alpha'], ['+', ['push-address', 'alpha', 'addr0'], 5, 2]]
args [['+', ['push-address', 'alpha', 'addr0'], 5, 2]]
x ['+', ['push-address', 'alpha', 'addr0'], 5, 2]
in else. x ['+', ['push-address', 'alpha', 'addr0'], 5, 2]
x +
x 5
x 2
vals ['', tensor(5), tensor(2)]
in primitives <function add_primitive at 0x1316c8040> ['', tensor(5), tensor(2)]


tensor(7)

In [46]:
from old_primitives import penv
from torch import tensor

vals = ['',tensor(1.),tensor(2.)]
penv['+'](vals[1:])

tensor(3.)

In [ ]:
from tests import is_tol

assert(is_tol(ret, truth))